In [ ]:
from itertools import product

import numpy as np
import pandas as pd

import plotly.graph_objects as go

from IPython.display import display

from sympy import symbols, Eq, N, simplify, expand, S
from sympy import zeros, diag, Matrix

import qlp.eqn_converter as qlp

# New maximization condition

$$
    f(x_1, x_2) = 8 -(x_1 - 2)^2 - (x_2 - 2)^2 = \psi_\alpha M_{\alpha \beta} \psi_\beta
$$
with
$$ 
    M_{\alpha \beta} 
    = 
    - Q_{1\alpha}Q_{1\beta} 
    - Q_{2\alpha}Q_{2\beta} 
    + 4Q_{1\alpha}\delta_{\alpha\beta}
    + 4Q_{2\alpha}\delta_{\alpha\beta}
$$
and
$$
    \begin{pmatrix}
        x_1 \\ x_2
    \end{pmatrix}
    =
    Q
    \vec \psi \, ,
$$
where $\vec \psi$ is a vector in a bit basis such that $\psi_\alpha \in \{0,1\} \Rightarrow \psi^2_\alpha = \psi_\alpha$.

## Preparation

In [ ]:
def f(x1, x2):
    return 8 - (x1 - 2) ** 2 - (x2 - 2) ** 2


zmin = 6.0

xx, yy = np.meshgrid(*[np.linspace(0, 4, 100)] * 2)
zz = f(xx, yy)
zz = np.where(zz < zmin, np.nan, zz)

In [ ]:
f_data = go.Heatmap(
    x=np.linspace(0, 4, 100),
    y=np.linspace(0, 4, 100),
    z=zz,
    colorscale="Blues",
    zmin=zmin,
    zsmooth="best",
)
layout = go.Layout(
    paper_bgcolor="white", plot_bgcolor="white", height=600, width=600, showlegend=False
)

## Plot

In [ ]:
go.Figure(data=[f_data], layout=layout).show()

## Maximization without constraints

Let's take a 2 bit basis, so that $M$ is a $4\times4$ matrix.

In [ ]:
q = qlp.get_bit_map(2, 2)

print(q)

m = -q.T @ q + 4 * np.diag(q[0] + q[1])

m

In [ ]:
f_max = -10
psi_opt = None

data = []

for psi in product(*[range(2)] * q.shape[1]):
    f_classical = f(*q @ psi)
    f_quantum = psi @ m @ psi

    assert f_classical == f_quantum

    data.append({"x1": q[0, :] @ psi, "x2": q[1, :] @ psi, "f": f_classical})

    if f_classical > f_max:
        f_max = f_classical
        psi_opt = psi

assert f_max == 8
assert (q @ psi_opt == np.array([2, 2])).all()

pd.DataFrame(data).sort_values("f", ascending=False).head()

## Including constraints

The first constraint is a linear constraint in the sense that it will not affect the location of the minimum.

In [ ]:
x1, x2, s = symbols("x1 x2 s")
m = -1
b = 4

eq = m * x1 + b >= x2
eq

In [ ]:
x = np.linspace(0, 4, 5)
y = np.array([eq.lhs.subs({"x1": xx}) for xx in x], dtype=float)
contraint1_data = go.Scatter(
    x=x, y=y, line={"dash": "solid", "color": "black"}, mode="lines", text="Constraint"
)
max_data1 = go.Scatter(x=[2], y=[2], marker={"color": "green", "size": 15})

In [ ]:
go.Figure(data=[f_data, contraint1_data, max_data1], layout=layout,).update_layout(
    title={
        "text": f"Constrained: {eq}",
        "xanchor": "center",
        "yanchor": "top",
        "x": 0.5,
    }
).show()

## Implementation of constrained

In [ ]:
print("constrained")
display(eq)

constrained = eq.lhs - eq.rhs - s
print("constrained with slack)")
display(Eq(constrained, 0))

In [ ]:
Nx = 2
Ns = 1

xi = Matrix([S(f"x{i}") for i in range(Nx)] + [S(f"s{i}") for i in range(Ns)])
xi_to_x = np.diag(([1 for i in range(Nx)] + [0 for i in range(Ns)]))

print("xi")
display(xi)

print("xi_to_x@xi")
display(xi_to_x @ xi)

In [ ]:
alpha = Matrix([-1, -1, -1]).T
print("alpha")
display(alpha)

beta = Matrix([b])
print("beta")
display(beta)


print("alpha@xi + beta")
display(alpha @ xi + beta)

In [ ]:
Nvars = Nx + Ns
Nbits = 2

q = qlp.get_bit_map(3, 2)
print(q)

psi = Matrix([f"psi_{i}{j}" for i in range(Nvars) for j in range(Nbits)])
print("psi")
display(psi)

print("q@psi")
display(Eq(xi, q @ psi))

print("alpha@q@psi")
display(alpha @ q @ psi)

In [ ]:
constrained_mat = np.array(
    q.T @ alpha.T @ alpha @ q + diag(*beta.T @ alpha @ q) + diag(*q.T @ alpha.T @ beta)
)
constrained_mat

### Cross Checks

In [ ]:
r1 = expand((alpha @ q @ psi + beta) ** 2)[0, 0]
r2 = expand(psi.T @ constrained_mat @ psi + beta.T @ beta)[0, 0]

diff = simplify(r1 - r2)
print("(alpha@q@psi + beta)**2 - psi.T@Omega@psi - beta.T@beta")
display(diff)

print("Diff after psi_ij**2 -> psi_ij")
display(diff.subs({S(symb) ** 2: symb for symb in diff.free_symbols}))

In [ ]:
data = []

Alpha = np.array(alpha)
Beta = np.array(beta)

for psi_vec in product(*[range(2)] * len(psi)):
    xi_vec = q @ psi_vec
    f_classical = ((Alpha @ xi_vec + Beta).T @ (Alpha @ xi_vec + Beta))[0, 0]
    f_quantum = psi_vec @ constrained_mat @ psi_vec

    data.append(
        {
            "x0": xi_vec[0],
            "x1": xi_vec[1],
            "s0": xi_vec[2],
            "f_classical": f_classical,
            "f_quantum": f_quantum,
        }
    )

df = pd.DataFrame(data).sort_values("f_classical", ascending=True)
df["diff"] = df["f_classical"] - df["f_quantum"]
df.head(5)

In [ ]:
print("Is there a constant difference between classical and qunatum which is b**2?")
assert all(df["diff"].unique() == b ** 2)
print("-> Yes!")

In [ ]:
print("Are all classical solutions larger or equal to zero?")
assert all(df.f_classical.unique() >= 0)
print("-> Yes!")

In [ ]:
print("Do all solutions which are equal to zero fulfill the constrained?")
assert (
    df.query("f_classical == 0")[["x0", "x1"]]
    .apply(lambda row: eq.subs({"x1": row["x0"], "x2": row["x1"]}), axis=1)
    .astype(bool)
    .all()
)
print("-> Yes!")

In [ ]:
print("Do all solutions which are larger than zero a violation of the constrained?")
assert not (
    df.query("f_classical > 0")[["x0", "x1"]]
    .apply(lambda row: eq.subs({"x1": row["x0"], "x2": row["x1"]}), axis=1)
    .astype(bool)
    .all()
)
print("-> Yes!")

## Implementation of optimizer

In [ ]:
m = (-q.T @ xi_to_x.T @ xi_to_x @ q + 4 * np.diag((xi_to_x @ q)[0] + (xi_to_x @ q)[1]))
m

In [ ]:
p =10
Omega = m - p * constrained_mat

In [ ]:
data = []

Alpha = np.array(alpha)
Beta = np.array(beta)

for psi_vec in product(*[range(2)] * len(psi)):
    xi_vec = q @ psi_vec
    f_classical = float(f(xi_vec[0], xi_vec[1]) - p*((Alpha @ xi_vec + Beta).T @ (Alpha @ xi_vec + Beta))[0, 0])
    f_quantum = float(psi_vec @ Omega @ psi_vec)

    data.append(
        {
            "x0": xi_vec[0],
            "x1": xi_vec[1],
            "s0": xi_vec[2],
            "f_classical": f_classical,
            "f_quantum": f_quantum,
        }
    )

df = pd.DataFrame(data).sort_values("f_classical", ascending=False)
df["diff"] = df["f_classical"] - df["f_quantum"]

assert df.f_classical.idxmax() == df.f_quantum.idxmax()
assert df.f_classical.max() == df.f_quantum.max() + df["diff"].unique()

print("Best solution:", df.loc[df.f_quantum.idxmax(), ["x0", "x1", "s0"]].values)

df.head(5)

In [ ]:
print("Is there a constant difference between classical and qunatum which is b**2?")
assert all(df["diff"].unique() == - p * b ** 2)
print("-> Yes!")